In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
ostro_dir = 'Osteosarcoma-UT'



# Define image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [4]:
from sklearn.model_selection import train_test_split
import os

# Define the root directory containing subfolders for each class
root_dir = 'Osteosarcoma-UT'

# Get the list of class names
classes = os.listdir(root_dir)
print(classes)

# Split the data into training and validation sets
train_data = []
val_data = []
for cls in classes:
    cls_dir = os.path.join(root_dir, cls)
    images = [os.path.join(cls_dir, img) for img in os.listdir(cls_dir)]
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    train_data.extend([(img, cls) for img in train_images])
    val_data.extend([(img, cls) for img in val_images])

# Further split the training data into training and testing sets
train_images, test_images = train_test_split(train_data, test_size=0.2, random_state=42)

# Now you have train_images, val_data, and test_images containing paths to images for each set



['Viable', 'Non-Tumor', 'Non-Viable-Tumor']


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame with two columns: 'file_path' and 'label'
train_df = pd.DataFrame(train_data, columns=['file_path', 'label'])
val_df = pd.DataFrame(val_data, columns=['file_path', 'label'])
test_df = pd.DataFrame(test_images, columns=['file_path', 'label'])

# Split the data into file paths and labels
train_data = train_df['file_path']
train_labels = train_df['label']

val_data = val_df['file_path']
val_labels = val_df['label']

test_data = test_df['file_path']
test_labels = test_df['label']

# You can create separate data generators for each set
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 914 validated image filenames belonging to 3 classes.
Found 230 validated image filenames belonging to 3 classes.
Found 183 validated image filenames belonging to 3 classes.


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD


img_width, img_height = 224, 224
batch_size = 32

base_model = ResNet50(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # You can add more dense layers if needed
predictions = Dense(3, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=SGD(lr=0.001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [8]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=[early_stopping])

Epoch 1/100


2024-02-21 16:24:09.694130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-02-21 16:24:09.790275: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp.


29/29 [==============================] - 21s 569ms/step - loss: 3.8976 - accuracy: 0.3775 - val_loss: 5.7269 - val_accuracy: 0.2565
Epoch 2/100
29/29 [==============================] - 16s 538ms/step - loss: 16.1936 - accuracy: 0.3523 - val_loss: 36.7301 - val_accuracy: 0.2565
Epoch 3/100
29/29 [==============================] - 16s 538ms/step - loss: 36.5495 - accuracy: 0.3665 - val_loss: 14.3394 - val_accuracy: 0.4652
Epoch 4/100
29/29 [==============================] - 16s 536ms/step - loss: 28.7940 - accuracy: 0.3414 - val_loss: 39.3776 - val_accuracy: 0.2783
Epoch 5/100
29/29 [==============================] - 16s 536ms/step - loss: 33.6041 - accuracy: 0.3512 - val_loss: 36.9460 - val_accuracy: 0.5217
Epoch 6/100
29/29 [==============================] - 16s 543ms/step - loss: 27.8768 - accuracy: 0.3928 - val_loss: 25.9222 - val_accuracy: 0.4652


In [9]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

5/5 [==============================] - 2s 301ms/step - loss: 5.8027 - accuracy: 0.2500
Test accuracy: 0.25


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score

# Assuming you already have your test generator and model trained

# Evaluate the model using test generator
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

# Get predictions and true labels
# Ensure that steps is set to get the correct number of samples
y_pred = model.predict_generator(test_generator, steps=test_generator.samples // batch_size + 1)
y_true = test_generator.classes

# Only take the necessary number of samples from y_true
y_true = y_true[:len(y_pred)]

# Calculate macro-averaged F1-score
macro_f1 = f1_score(y_true, y_pred.argmax(axis=1), average='macro')
print("Macro-averaged F1-score:", macro_f1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred.argmax(axis=1))
print("Confusion matrix:\n", cm)

# Calculate AUC-ROC score for each class
y_onehot = np.zeros((len(y_true), len(np.unique(y_true))))
y_onehot[np.arange(len(y_true)), y_true] = 1
auc_roc_scores = roc_auc_score(y_onehot, y_pred, multi_class='ovr')
print("AUC-ROC scores for each class:", auc_roc_scores)

# Calculate average AUC-ROC score
average_auc_roc = np.mean(auc_roc_scores)
print("Average AUC-ROC score:", average_auc_roc)
